In [1]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

ССылки на файлы

In [2]:
ASU_dataset = Path('data/train_dataset_train.csv')

Загрузка баового датасета

In [3]:

asu_frame = pd.read_csv(ASU_dataset)
asu_frame.head(2)

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Общежитие,Наличие_Матери,Наличие_Отца,Страна_Родители,Опекунство,Село,Иностранец,КодФакультета,СрБаллАттестата,Статус
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,0.0,0,0.0,Россия,0.0,0.0,0.0,41.0,4.294,3
1,63306,14895,2010,Муж,ЦН,NaN,1992-04-18 00:00:00.000,NaN,NaN,NaN,...,0.0,1,1.0,Россия,0.0,0.0,0.0,28.0,67.000,4


In [4]:
asu_frame.shape

(13584, 24)

Очистка значений датафрейма

In [5]:
asu_frame.loc[asu_frame['Пол'] == 'муж', 'Пол'] = 'Муж'

In [6]:
asu_frame['male'] = 0
asu_frame['female'] = 0

asu_frame.loc[asu_frame['Пол'] == 'Муж', 'male'] = 1
asu_frame.loc[asu_frame['Пол'] == 'Жен', 'female'] = 1
asu_frame['sex_summ'] = asu_frame['male'] + asu_frame['female']
asu_frame.loc[asu_frame['sex_summ'] != 1].shape[0]

7

In [7]:
#asu_frame['birth_year'] = asu_frame['Дата_Рождения'].dt.to_period('Y')
asu_frame['Дата_Рождения']
asu_frame['birth_year'] = pd.to_datetime(asu_frame['Дата_Рождения'])
asu_frame['birth_year'].dt.to_period('Y')
asu_frame['birth_year'].all()

asu_frame['entrance'] = pd.to_datetime(asu_frame['Год_Поступления'], format='%Y')
asu_frame['entrance'].sort_values()



4719    2001-01-01
10232   2003-01-01
8941    2004-01-01
9417    2005-01-01
7300    2007-01-01
           ...    
10228   2020-01-01
5126    2020-01-01
2748    2020-01-01
12961   2020-01-01
104     2212-01-01
Name: entrance, Length: 13584, dtype: datetime64[ns]

In [8]:
asu_frame['birth_year_int'] = asu_frame['Дата_Рождения'].str.slice(start=0, stop=4).astype('int64')
asu_frame['birth_year_int']

0        1998
1        1992
2        1999
3        1994
4        1995
         ... 
13579    1998
13580    1997
13581    1998
13582    1995
13583    1995
Name: birth_year_int, Length: 13584, dtype: int64

In [9]:
frame_size = asu_frame.shape
frame_size

(13584, 30)

In [10]:
asu_frame.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Пол', 'Основания',
       'Изучаемый_Язык', 'Дата_Рождения', 'Уч_Заведение', 'Где_Находится_УЗ',
       'Год_Окончания_УЗ', 'Пособие', 'Страна_ПП', 'Регион_ПП', 'Город_ПП',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance',
       'birth_year_int'],
      dtype='object')

In [11]:
asu_frame['Статус']

0        3
1        4
2        4
3        4
4        4
        ..
13579    3
13580    4
13581    4
13582    4
13583    4
Name: Статус, Length: 13584, dtype: int64

In [12]:
asu_frame.groupby('Статус')['СрБаллАттестата'].describe()

,count,mean,std,min,25%,50%,75%,max
Статус,,,,,,,,
-1,614.0,100.811304,429.151037,3.285,48.0,61.0,71.0,4800.0
3,4721.0,82.513324,344.527967,0.000,44.0,56.0,70.0,5000.0
4,8249.0,65.853001,166.979956,0.000,49.0,64.0,79.0,5000.0


In [13]:
asu_frame.groupby('Статус')['Село'].value_counts()

Статус  Село
-1      0.0      504
        1.0      110
 3      0.0     3539
        1.0     1132
 4      0.0     6358
        1.0     1855
Name: Село, dtype: int64

In [14]:
asu_frame.pivot_table(index='Пол', columns='Статус')

ID                             birth_year_int               \
Статус            -1             3             4             -1            3   
Пол                                                                            
Жен     83378.066667  71829.522481  68025.234389    1995.407407  1993.657785   
Муж     83746.534884  71596.500216  68424.950090    1996.889535  1994.087257   

                    female       male  ... Опекунство Пособие            \
Статус            4     -1  3  4   -1  ...          4      -1    3    4   
Пол                                    ...                                
Жен     1993.971434      1  1  1    0  ...   0.000183     0.0  0.0  0.0   
Муж     1993.848115      0  0  0    1  ...   0.000000     0.0  0.0  0.0   

            Село                     СрБаллАттестата                        
Статус        -1         3         4              -1          3          4  
Пол                                                                         
Жен     0.196296  0.265487  0.243099       72.675426  93.906158  65.087485  
Муж     0.165698  0.218832  0.192363      122.894696  70.760177  67.357506  

[2 rows x 51 columns]

In [15]:
graduate = asu_frame.loc[asu_frame['Статус'] == 4]['Статус'].count()
expelled = asu_frame.loc[asu_frame['Статус'] == 3]['Статус'].count()
vacation = asu_frame.loc[asu_frame['Статус'] == -1]['Статус'].count()
all_students = frame_size[0]
delta = all_students - graduate - expelled - vacation
f'Всего позиций {all_students} закончили ВУЗ {graduate}, отчислено {expelled}, каникулы {vacation}. Отклонение по сумме {delta}'

'Всего позиций 13584 закончили ВУЗ 8249, отчислено 4721, каникулы 614. Отклонение по сумме 0'

In [16]:
graduate_frame = asu_frame.loc[asu_frame['Статус'] == 4]
graduate_frame

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int
1,63306,14895,2010,Муж,ЦН,NaN,1992-04-18 00:00:00.000,NaN,NaN,NaN,...,0.0,28.0,67.000,4,1,0,1,1992-04-18,2010-01-01,1992
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,0.0,41.0,4.133,4,0,1,1,1999-03-21,2017-01-01,1999
3,81661,19838,2017,Жен,БН,Английский язык,1994-12-08 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,35.0,74.000,4,0,1,1,1994-12-08,2017-01-01,1994
4,81509,19803,2017,Жен,БН,Английский язык,1995-06-28 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,28.0,53.000,4,0,1,1,1995-06-28,2017-01-01,1995
6,57717,16945,2013,Муж,СН,NaN,1995-05-10 00:00:00.000,NaN,NaN,NaN,...,0.0,51.0,67.000,4,1,0,1,1995-05-10,2013-01-01,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13578,56928,17038,2013,Жен,СН,Английский язык,1989-12-21 00:00:00.000,"Автономная некоммерческая организация ВПО ""Алт...","г. Барнаул, Алтайский край",2011.0,...,0.0,26.0,51.000,4,0,1,1,1989-12-21,2013-01-01,1989
13580,71516,19433,2015,Жен,ОО,Немецкий язык,1997-09-19 00:00:00.000,"МКОУ ""Георгиевская СОШ""","Алтайский край, Ребрихинский р-н, Георгиевка с",2015.0,...,0.0,26.0,82.000,4,0,1,1,1997-09-19,2015-01-01,1997
13581,76970,18723,2016,Муж,СН,Английский язык,1998-03-20 00:00:00.000,МБОУ СОШ №113 им. Сергея Семенова,"Россия, Алтайский край, г Барнаул",2016.0,...,0.0,41.0,3.400,4,1,0,1,1998-03-20,2016-01-01,1998
13582,48548,14663,2012,Жен,ОО,NaN,1995-07-06 00:00:00.000,NaN,NaN,NaN,...,0.0,24.0,95.000,4,0,1,1,1995-07-06,2012-01-01,1995


In [17]:
def doly(x):
    
    gh = x['ID'].count() / frame_size[0]
    print(gh)
    return gh

asu_frame.groupby(['Пол', 'Статус']).apply(doly)

0.01987632508833922
0.1768256772673734
0.40201707891637223
0.025323910482921083
0.1704210836277974
0.2050206124852768


Пол  Статус
Жен  -1        0.019876
      3        0.176826
      4        0.402017
Муж  -1        0.025324
      3        0.170421
      4        0.205021
dtype: float64

In [18]:
asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()

/tmp/ipykernel_67900/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()
/tmp/ipykernel_67900/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()
/tmp/ipykernel_67900/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of

,count,unique,top,freq,first,last
birth_year,,,,,,
1964,1,1,2016-01-01 00:00:00,1,2016-01-01 00:00:00,2016-01-01 00:00:00
1965,1,1,2014-01-01 00:00:00,1,2014-01-01 00:00:00,2014-01-01 00:00:00
1966,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00
1969,1,1,2014-01-01 00:00:00,1,2014-01-01 00:00:00,2014-01-01 00:00:00
1971,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00
1972,3,3,2013-01-01 00:00:00,1,2013-01-01 00:00:00,2016-01-01 00:00:00
1973,2,2,2016-01-01 00:00:00,1,2015-01-01 00:00:00,2016-01-01 00:00:00
1974,1,1,2016-01-01 00:00:00,1,2016-01-01 00:00:00,2016-01-01 00:00:00
1975,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00


In [19]:
old_year = datetime.datetime.strptime('1986', '%Y')
asu_frame.loc[(asu_frame['birth_year'] < old_year) & (asu_frame['СрБаллАттестата'] > 20)]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int
17,53187,12426,2012,Муж,ДН,NaN,1982-06-09 00:00:00.000,NaN,NaN,NaN,...,NaN,26.0,88.0,3,1,0,1,1982-06-09,2012-01-01,1982
21,58638,16973,2012,Жен,ОО,NaN,1976-09-20 00:00:00.000,NaN,NaN,NaN,...,0.0,48.0,63.0,4,0,1,1,1976-09-20,2012-01-01,1976
47,56984,12412,2013,Муж,ДН,Английский язык,1980-11-07 00:00:00.000,Алтайский государственный университет,"г.Барнаул, Алтайский край",2003.0,...,NaN,26.0,80.0,3,1,0,1,1980-11-07,2013-01-01,1980
52,67548,15633,2014,Жен,ОО,Английский язык,1967-02-07 00:00:00.000,"ФГБОУ ВПО ""Алтайский государственный университет""","Алтайский край, Барнаул г",1992.0,...,0.0,31.0,80.0,3,0,1,1,1967-02-07,2014-01-01,1967
53,59684,12812,2013,Муж,ДН,Английский язык,1985-11-19 00:00:00.000,"ГОУ СПО СО ""Каменск-Уральский техникум торговл...",г. Каменск-Уральский,2010.0,...,NaN,45.0,85.0,3,1,0,1,1985-11-19,2013-01-01,1985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13539,80890,21005,2017,Жен,ОО,Английский язык,1976-09-22 00:00:00.000,Всероссийский заочный финансово-экономический ...,"Москва г,",1998.0,...,0.0,27.0,39.0,3,0,1,1,1976-09-22,2017-01-01,1976
13556,65615,15431,2014,Жен,ОО,Немецкий язык,1970-08-25 00:00:00.000,"НГУ ""Современная гуманитарная академия""","Московская обл, Москва г",2006.0,...,0.0,34.0,95.0,4,0,1,1,1970-08-25,2014-01-01,1970
13559,65958,18141,2014,Жен,ОО,Английский язык,1985-07-21 00:00:00.000,"МБОУ ""Сибирская СОШ""","Алтайский край, Первомайский р-н, Сибирский п",2002.0,...,0.0,25.0,88.0,3,0,1,1,1985-07-21,2014-01-01,1985
13566,56942,14815,2013,Жен,ОО,Английский язык,1980-07-06 00:00:00.000,"АНО ВПО ""Алтайская академия экономики и права ...",Алтайский край г. Барнаул,2002.0,...,NaN,26.0,84.0,4,0,1,1,1980-07-06,2013-01-01,1980


In [20]:
old_year = datetime.datetime.strptime('2004', "%Y")


In [21]:
asu_frame.groupby('Код_группы')['ID'].count().max()

23

# Очистка первичного фрейма

In [22]:
clean_frame = pd.DataFrame()

In [23]:
clean_frame = asu_frame.copy()

Колонка иностранный язык

In [24]:
clean_frame.loc[clean_frame['Изучаемый_Язык'].isnull(), 'Изучаемый_Язык'] = '-'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Англиийский', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Английский)', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Немецкий)', 'Изучаемый_Язык'] = 'Немецкий язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Русский язык', 'Изучаемый_Язык'] = '-'
clean_frame.groupby('Изучаемый_Язык')['ID'].count()

Изучаемый_Язык
-                     837
Английский язык     10470
Немецкий язык        2223
Французский язык       54
Name: ID, dtype: int64

In [25]:
clean_frame.loc[clean_frame['Пол'].isnull(), 'Пол'] = '-'
clean_frame.groupby('Пол')['ID'].count()

Пол
-         7
Жен    8133
Муж    5444
Name: ID, dtype: int64

In [26]:
clean_frame.loc[clean_frame['Уч_Заведение'].isnull(), 'Уч_Заведение'] = '-'
clean_frame.groupby('Уч_Заведение')['ID'].count()

Уч_Заведение
-                                                           1233
АГАУ                                                           4
АГУ СПО                                                        1
АКПЛ                                                           1
АНО "Горно-Алтайский колледж экономики и права"                2
                                                            ... 
школа иностранных языков "Хуавэй" провинции Чжэцзян            1
школа-гимназия №1 им. Михаила Бабкина                          1
школа-гимназия №5 им. Бекмамата Осмонова гор. Жалал-Абад       2
экспериментальная средняя школа Инкоу г                        2
экспериментальная средняя школа провинции Цзилинь              1
Name: ID, Length: 3647, dtype: int64

In [27]:
clean_frame.loc[clean_frame['Где_Находится_УЗ'].isnull(), 'Где_Находится_УЗ'] = '-'

In [28]:
clean_frame.loc[clean_frame['Пособие'].isnull(), 'Пособие'] = 0
clean_frame.groupby('Пособие')['ID'].count()

Пособие
0.0    13584
Name: ID, dtype: int64

In [29]:
clean_frame.loc[clean_frame['Страна_ПП'].isnull(), 'Страна_ПП'] = '-'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Киргизия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызстан', 'Страна_ПП'] = 'Кыргызская Республика'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан ВКО', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан Респ', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан респ', 'Страна_ПП'] = 'Республика Казахстан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан', 'Страна_ПП'] = 'Республика Таджикистан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан Респ', 'Страна_ПП'] = 'Республика Таджикистан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'РОССИЯ', 'Страна_ПП'] = 'Россия'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Росссия', 'Страна_ПП'] = 'Россия'
clean_frame.groupby('Страна_ПП')['ID'].count()

Страна_ПП
-                           347
Армения                       2
Китай                       148
Кыргызская Республика        61
Монголия                      2
Нигерия                       1
Республика Казахстан        423
Республика Таджикистан      121
Россия                    12474
Туркменистан                  2
Узбекистан                    1
Украина                       1
Франция                       1
Name: ID, dtype: int64

In [30]:
clean_frame.loc[clean_frame['Регион_ПП'].isnull(), 'Регион_ПП'] = '-'

In [31]:
clean_frame.loc[clean_frame['Город_ПП'].isnull(), 'Город_ПП'] = '-'

In [32]:
clean_frame.loc[clean_frame['Общежитие'].isnull(), 'Общежитие'] = 0
clean_frame.groupby('Общежитие')['ID'].count()

Общежитие
0.0    10616
1.0     2968
Name: ID, dtype: int64

In [33]:
clean_frame.loc[clean_frame['Страна_Родители'].isnull(), 'Страна_Родители'] = '-'

clean_frame.loc[clean_frame['Страна_Родители'] == 'КНР', 'Страна_Родители'] = 'Китай'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Казахстан', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'казахстан', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Кыргызстан', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Киргизия', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Таджикистан', 'Страна_Родители'] = 'Республика Таджикистан'
clean_frame.groupby('Страна_Родители')['ID'].count()

Страна_Родители
-                           458
Армения                       3
Германия                      1
Китай                       148
Кыргызская Республика        71
Монголия                      2
Нигерия                       1
Республика Казахстан        516
Республика Молдова            1
Республика Таджикистан      135
Россия                    12239
Туркменистан                  5
Узбекистан                    3
Украина                       1
Name: ID, dtype: int64

In [34]:
clean_frame.loc[clean_frame['Опекунство'].isnull(), 'Опекунство'] = 0
clean_frame.groupby('Опекунство')['ID'].count()

Опекунство
0.0    13578
1.0        6
Name: ID, dtype: int64

In [35]:

clean_frame.loc[(clean_frame['Село'].isnull()) & (~clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 1
clean_frame.loc[(clean_frame['Село'].isnull()) & (clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 0
clean_frame.loc[(clean_frame['Село'].isnull())]
clean_frame.groupby('Село')['ID'].count()

Село
0.0    10425
1.0     3159
Name: ID, dtype: int64

In [36]:

clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (~clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 1
clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 0
clean_frame.loc[(clean_frame['Иностранец'].isnull())]
clean_frame.groupby('Иностранец')['ID'].count()

Иностранец
0.0    12337
1.0     1247
Name: ID, dtype: int64

In [37]:
clean_frame.loc[(clean_frame['КодФакультета'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [38]:
clean_frame.loc[(clean_frame['СрБаллАттестата'].isnull())]
big_atestat = clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'ID'].to_list()
small_atestat = clean_frame.loc[clean_frame['СрБаллАттестата'] < 5, 'ID'].to_list()
clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'СрБаллАттестата'] = clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'СрБаллАттестата'] / 100
clean_frame.loc[clean_frame['СрБаллАттестата'] < 5, 'СрБаллАттестата'] = clean_frame.loc[clean_frame['СрБаллАттестата'] < 5, 'СрБаллАттестата'] * 10
clean_frame.loc[clean_frame['ID'].isin(big_atestat), 'СрБаллАттестата']




311      46.66
342      32.50
411      48.66
730      45.33
1269     42.00
1875     42.14
1956     41.53
1981     43.33
2263     40.00
2388     37.69
2405     44.28
2538     43.57
2803     36.67
4017     30.67
4296     46.00
4723     47.69
4924     32.67
6604     44.00
6814     48.70
7003     43.33
7848     48.00
7902     44.21
7963     38.75
7990     40.00
8288     46.67
8319     41.87
8483     42.67
8910     32.31
8945     38.66
8977     39.17
9101     44.00
9233     40.00
9698     50.00
9795     50.00
9995     50.00
10013    49.47
10095    43.33
10216    40.76
10228    40.66
10317     8.25
10391    42.50
10456    42.50
10848    35.62
11424    22.50
11762    43.75
12076    41.18
12770    40.76
12830    45.00
12880    40.00
12961    47.69
13283    42.00
13306    41.67
13338    42.67
13512    36.47
Name: СрБаллАттестата, dtype: float64

In [39]:
clean_frame.loc[(clean_frame['Статус'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [40]:
clean_frame['СрБаллАттестата'].describe()

count    13584.000000
mean        61.723057
std         18.054371
min          0.000000
25%         47.000000
50%         61.000000
75%         75.000000
max        100.000000
Name: СрБаллАттестата, dtype: float64

In [41]:
asu_frame.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Пол', 'Основания',
       'Изучаемый_Язык', 'Дата_Рождения', 'Уч_Заведение', 'Где_Находится_УЗ',
       'Год_Окончания_УЗ', 'Пособие', 'Страна_ПП', 'Регион_ПП', 'Город_ПП',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance',
       'birth_year_int'],
      dtype='object')

In [42]:
columns_to_csv = ['Год_Поступления', 'Пол', 'Основания', 'Изучаемый_Язык', 'Дата_Рождения', 'Страна_ПП', 'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance']

In [43]:
clean_frame.columns[clean_frame.isna().any()].tolist()

['Год_Окончания_УЗ']

In [44]:
clean_frame.loc[:, ['Основания', 'male', 'female']].head(10)

,Основания,male,female
0,ОО,0,1
1,ЦН,1,0
2,ДН,0,1
3,БН,0,1
4,БН,0,1
5,СН,1,0
6,СН,1,0
7,СН,0,1
8,СН,0,1
9,ОО,1,0


# Составление справочников кодов для текстовых колонок

In [45]:
input_train = clean_frame.head(10).copy()
input_train.head(2)

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,0.0,41.0,42.94,3,0,1,1,1998-02-08,2015-01-01,1998
1,63306,14895,2010,Муж,ЦН,-,1992-04-18 00:00:00.000,-,-,NaN,...,0.0,28.0,67.00,4,1,0,1,1992-04-18,2010-01-01,1992


In [46]:
clean_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13584 entries, 0 to 13583
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                13584 non-null  int64         
 1   Код_группы        13584 non-null  int64         
 2   Год_Поступления   13584 non-null  int64         
 3   Пол               13584 non-null  object        
 4   Основания         13584 non-null  object        
 5   Изучаемый_Язык    13584 non-null  object        
 6   Дата_Рождения     13584 non-null  object        
 7   Уч_Заведение      13584 non-null  object        
 8   Где_Находится_УЗ  13584 non-null  object        
 9   Год_Окончания_УЗ  12284 non-null  float64       
 10  Пособие           13584 non-null  float64       
 11  Страна_ПП         13584 non-null  object        
 12  Регион_ПП         13584 non-null  object        
 13  Город_ПП          13584 non-null  object        
 14  Общежитие         1358

In [47]:
# проверка полноты формирования спика колонок по форматам
non_digit_columns_num = clean_frame.dtypes[(clean_frame.dtypes == "datetime64[ns]") | (clean_frame.dtypes == "object")].count()
digit_columns_num = clean_frame.dtypes[(clean_frame.dtypes == "int64") | (clean_frame.dtypes == "float64")].count()
clean_frame.shape[1] - (non_digit_columns_num + digit_columns_num)

0

In [48]:
# список не числовых колонок
non_digit_columns = clean_frame.dtypes[(clean_frame.dtypes == "datetime64[ns]") | (clean_frame.dtypes == "object")].index.to_list()
non_digit_columns

['Пол',
 'Основания',
 'Изучаемый_Язык',
 'Дата_Рождения',
 'Уч_Заведение',
 'Где_Находится_УЗ',
 'Страна_ПП',
 'Регион_ПП',
 'Город_ПП',
 'Страна_Родители',
 'birth_year',
 'entrance']

In [49]:
# список числовых колонок

digit_columns = clean_frame.dtypes[(clean_frame.dtypes == "int64") | (clean_frame.dtypes == "float64")].index.to_list()
digit_columns

['ID',
 'Код_группы',
 'Год_Поступления',
 'Год_Окончания_УЗ',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'Статус',
 'male',
 'female',
 'sex_summ',
 'birth_year_int']

In [50]:
columns_to_code = {
    'Основания': 'basis',
    'Изучаемый_Язык': 'language',    
    'Страна_ПП': 'country',
    'Регион_ПП': 'region',
    'Город_ПП': 'city',
    'Страна_Родители': 'parents_country',
}

In [51]:
def code_dict(column_name: str):
    code_frame = clean_frame.groupby(column_name)['ID'].count()
    code_list = code_frame.index.to_list()
    target_dict = {}
    for num, name in enumerate(code_list):
        target_dict[name] = num
    return target_dict

In [52]:
def test(x, arg):
    mes = f'chore - {x}, change to {arg[x]}'
    print(mes)

#input_train['Изучаемый_Язык'].apply(test, args=({'Английский язык': 'o', '-': 'g'},))

In [53]:
# Функция для преобразования слова в код по заданному правилу

def get_code(x: pd.Series, code_dict) -> pd.Series:    
    return code_dict[x]



In [54]:
codes_rule = {}
for name, code in columns_to_code.items():
    target_dict = code_dict(name)
    codes_rule[name] = target_dict
    input_train[columns_to_code[name]] = input_train[name].apply(get_code, args=(target_dict,))

input_train.head(3)

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,sex_summ,birth_year,entrance,birth_year_int,basis,language,country,region,city,parents_country
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,1,1998-02-08,2015-01-01,1998,2,1,8,14,74,10
1,63306,14895,2010,Муж,ЦН,-,1992-04-18 00:00:00.000,-,-,NaN,...,1,1992-04-18,2010-01-01,1992,4,0,8,14,74,10
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,1,1999-03-21,2017-01-01,1999,1,1,8,14,31,10


In [55]:
list(columns_to_code.values())

['basis', 'language', 'country', 'region', 'city', 'parents_country']

In [56]:
columns_to_model = digit_columns + list(columns_to_code.values())
columns_to_model

['ID',
 'Код_группы',
 'Год_Поступления',
 'Год_Окончания_УЗ',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'Статус',
 'male',
 'female',
 'sex_summ',
 'birth_year_int',
 'basis',
 'language',
 'country',
 'region',
 'city',
 'parents_country']

In [57]:
input_train.loc[:, columns_to_model].to_csv('data/clean.csv', index_label='ind')

In [58]:
codes_rule = {}
for name, code in columns_to_code.items():
    target_dict = code_dict(name)
    codes_rule[name] = target_dict    
    clean_frame[columns_to_code[name]] = clean_frame[name].apply(get_code, args=(code_dict(name),))

clean_frame.head(3)

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,sex_summ,birth_year,entrance,birth_year_int,basis,language,country,region,city,parents_country
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,1,1998-02-08,2015-01-01,1998,2,1,8,14,74,10
1,63306,14895,2010,Муж,ЦН,-,1992-04-18 00:00:00.000,-,-,NaN,...,1,1992-04-18,2010-01-01,1992,4,0,8,14,74,10
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,1,1999-03-21,2017-01-01,1999,1,1,8,14,31,10


In [59]:
frame_to_csv = clean_frame.copy()

In [60]:

frame_to_csv.loc[:, columns_to_model].to_csv('data/clean_frame.csv', index_label='ind')

In [61]:
import yaml
with open('data/codes.yaml', 'w') as file:
    yaml.dump(codes_rule, file)